In [255]:
import pandas as pd
import datetime as dt
import regex as re
import os
from pathlib import Path
import utils_laramie
import numpy as np

### Read in the data

In [205]:
def get_all_raw_data(dirpath = Path('raw_data/')):
    csv_list = [filename for filename in os.listdir(dirpath)]
    return csv_list

In [206]:
def clean_col_names(list_dfs, list_csvs):

    list_cols = [df.columns.to_list() for df in list_dfs]
    zipped = zip(list_csvs,list_cols)
    name_list = []
    column_list = []
    for tup in list(zipped):
        name = tup[0]
        match = re.match(r'(\w+)_',name)
        name = match.group(1)
        for col in tup[1]:
            if name.lower() != col.lower():
                name_list.append((name.upper()+'_'+col.lower()))
            else:
                name_list.append(name.upper())

    return name_list


In [207]:
def drop_unnamed(df):
    for col in df.columns:
        if 'Unnamed' in str(col):
            df.drop(columns= col, inplace=True)
    return df
    

In [208]:
def get_df(list_of_csvs= ['SPY_data.csv','TR_data.csv','VIX_data.csv']):
    """concat a list of csvs into a single df"""
    # make a list of dataframes
    list_dfs = [pd.read_csv(f'raw_data/{_csv}', parse_dates = True, infer_datetime_format = True) for _csv in list_of_csvs]
    for df in list_dfs:
        df.drop_duplicates(inplace=True)
        df['date'] = pd.to_datetime(df['date']).dt.date
        df['date'] = pd.to_datetime(df['date'])
        df = drop_unnamed(df)
    clean_headers = clean_col_names(list_dfs,list_of_csvs)
    merged_df = pd.concat(list_dfs, axis=1, join= 'inner')
    merged_df.columns = clean_headers
    return merged_df
x = get_df()
x 

,SPY_date,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR_date,TR,VIX_date,VIX_open,VIX_high,VIX_low,VIX_close
0,2015-12-01,209.42,210.8200,209.1100,210.68,97858418,337780,209.921410,2015-12-02,2.7700,1990-01-02,17.24,17.24,17.24,17.24
1,2015-12-02,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055,2015-12-03,4.3989,1990-01-03,18.19,18.19,18.19,18.19
2,2015-12-03,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936,2015-12-04,4.3900,1990-01-04,19.22,19.22,19.22,19.22
3,2015-12-04,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631,2015-12-07,2.5295,1990-01-05,20.11,20.11,20.11,20.11
4,2015-12-07,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128,2015-12-08,2.5090,1990-01-08,20.26,20.26,20.26,20.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588,2022-03-23,446.90,448.4900,443.7100,443.80,77982935,627281,446.058683,2022-03-24,6.7000,1996-04-15,17.54,17.54,16.82,16.83
1589,2022-03-24,445.82,450.5000,444.7600,450.48,61411569,526865,447.609014,2022-03-25,4.5500,1996-04-16,16.51,16.56,16.21,16.45
1590,2022-03-25,451.09,452.9800,448.4300,452.69,77199328,661033,451.047221,2022-03-28,5.8500,1996-04-17,16.50,17.30,16.36,17.07
1591,2022-03-28,452.04,455.9100,450.0600,455.91,68356914,620868,453.294974,2022-03-29,6.2300,1996-04-18,16.69,17.06,16.24,16.41


In [209]:
def get_df(list_of_csvs= ['SPY_data.csv','TR_data.csv','VIX_data.csv']):
    """concat a list of csvs into a single df"""
    list_dfs = [pd.read_csv(f'raw_data/{_csv}', parse_dates = True, infer_datetime_format = True) for _csv in list_of_csvs]
    for df in list_dfs:
        df.drop_duplicates(inplace=True)
        df['date'] = pd.to_datetime(df['date']).dt.date
        df['date'] = pd.to_datetime(df['date'])
        df = drop_unnamed(df)
        df.set_index('date',inplace=True)
    clean_headers = clean_col_names(list_dfs,list_of_csvs)
    merged_df = pd.concat(list_dfs, axis=1, join= 'inner')  
    merged_df.columns = clean_headers
    return merged_df
x = get_df()
x 

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close
date,,,,,,,,,,,,
2015-12-02,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055,2.7700,15.04,16.49,14.71,15.91
2015-12-03,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936,4.3989,15.87,19.35,15.86,18.11
2015-12-04,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631,4.3900,17.43,17.65,14.69,14.81
2015-12-07,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128,2.5295,15.65,17.18,15.58,15.84
2015-12-08,206.48,208.2890,205.7800,206.99,103372367,387782,206.966276,2.5090,17.69,18.33,16.52,17.60
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-23,446.90,448.4900,443.7100,443.80,77982935,627281,446.058683,5.8800,23.04,24.03,22.64,23.57
2022-03-24,445.82,450.5000,444.7600,450.48,61411569,526865,447.609014,6.7000,23.61,23.79,21.49,21.67
2022-03-25,451.09,452.9800,448.4300,452.69,77199328,661033,451.047221,4.5500,21.87,22.86,20.80,20.81


### Create Weekly Range

In [177]:
df = get_df(get_all_raw_data())
df['weekly_range'] = 0


In [178]:
# week_group = df.groupby(by = [df.index.year, df.index.month, df.index.week]).mean()
df.head(3)

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range
date,,,,,,,,,,,,,
2015-12-02,210.6,211.00,208.2300,208.54,108069059,367013,209.563055,2.7700,15.04,16.49,14.71,15.91,0
2015-12-03,208.9,209.15,204.7511,205.58,166224154,546768,206.878936,4.3989,15.87,19.35,15.86,18.11,0
2015-12-04,206.1,209.97,205.9300,209.66,192878747,556731,208.178631,4.3900,17.43,17.65,14.69,14.81,0


In [210]:
week_high = 0  
week_low = 999999
for index, row in df.iterrows():
    if df.loc[index, 'DayOfWeek'] == 'Monday':
        week_high = df.loc[index,'SPY_high']
        week_low = df.loc[index,'SPY_low']
    else: 
        if df.loc[index,'SPY_high']>week_high:
            week_high=df.loc[index,'SPY_high']
        if df.loc[index,'SPY_low']<week_low:
            week_low=df.loc[index,'SPY_low']
        if df.loc[index,'DayOfWeek'] == 'Friday':
            df.loc[index,'weekly_range'] = week_high - week_low

### Group by year and week, then split into x (features) and y (target)

In [214]:
week_group = df.groupby(by = [df.index.isocalendar().year, df.index.isocalendar().week,df.index.isocalendar().day]).mean()


SPY_open  SPY_high   SPY_low  SPY_close   SPY_volume  \
year week day                                                         
2015 49   3      210.60  211.0000  208.2300     208.54  108069059.0   
          4      208.90  209.1500  204.7511     205.58  166224154.0   
          5      206.10  209.9700  205.9300     209.66  192878747.0   
     50   1      209.20  209.7295  207.2000     208.27  102027111.0   
          2      206.48  208.2890  205.7800     206.99  103372367.0   

               SPY_trade_count    SPY_vwap      TR  VIX_open  VIX_high  \
year week day                                                            
2015 49   3           367013.0  209.563055  2.7700     15.04     16.49   
          4           546768.0  206.878936  4.3989     15.87     19.35   
          5           556731.0  208.178631  4.3900     17.43     17.65   
     50   1           374705.0  208.276128  2.5295     15.65     17.18   
          2           387782.0  206.966276  2.5090     17.69     18.33   

               VIX_low  VIX_close  weekly_range  
year week day                                    
2015 49   3      14.71      15.91        0.0000  
          4      15.86      18.11        0.0000  
          5      14.69      14.81        6.2489  
     50   1      15.58      15.84        0.0000  
          2      16.52      17.60        0.0000

In [288]:
idx = pd.IndexSlice
week_group.loc[idx[2015, 49]].index

Index([3, 4, 5], dtype='object', name='day')

In [ ]:

df_x = week_group.drop(columns= 'weekly_range')
df_y = pd.DataFrame(week_group['weekly_range'])
week_group.head()


In [324]:
week_group.reset_index()

,year,week,day,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range
0,2015,49,3,210.60,211.0000,208.2300,208.54,108069059.0,367013.0,209.563055,2.7700,15.04,16.49,14.71,15.91,0.0000
1,2015,49,4,208.90,209.1500,204.7511,205.58,166224154.0,546768.0,206.878936,4.3989,15.87,19.35,15.86,18.11,0.0000
2,2015,49,5,206.10,209.9700,205.9300,209.66,192878747.0,556731.0,208.178631,4.3900,17.43,17.65,14.69,14.81,6.2489
3,2015,50,1,209.20,209.7295,207.2000,208.27,102027111.0,374705.0,208.276128,2.5295,15.65,17.18,15.58,15.84,0.0000
4,2015,50,2,206.48,208.2890,205.7800,206.99,103372367.0,387782.0,206.966276,2.5090,17.69,18.33,16.52,17.60,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,2022,12,3,446.90,448.4900,443.7100,443.80,77982935.0,627281.0,446.058683,5.8800,23.04,24.03,22.64,23.57,0.0000
1588,2022,12,4,445.82,450.5000,444.7600,450.48,61411569.0,526865.0,447.609014,6.7000,23.61,23.79,21.49,21.67,0.0000
1589,2022,12,5,451.09,452.9800,448.4300,452.69,77199328.0,661033.0,451.047221,4.5500,21.87,22.86,20.80,20.81,12.3000
1590,2022,13,1,452.04,455.9100,450.0600,455.91,68356914.0,620868.0,453.294974,5.8500,22.14,23.33,19.54,19.63,0.0000


In [215]:
display(df_y.head())
display(df_x.head())

weekly_range
year week day              
2015 49   3          0.0000
          4          0.0000
          5          6.2489
     50   1          0.0000
          2          0.0000

SPY_open  SPY_high   SPY_low  SPY_close   SPY_volume  \
year week day                                                         
2015 49   3      210.60  211.0000  208.2300     208.54  108069059.0   
          4      208.90  209.1500  204.7511     205.58  166224154.0   
          5      206.10  209.9700  205.9300     209.66  192878747.0   
     50   1      209.20  209.7295  207.2000     208.27  102027111.0   
          2      206.48  208.2890  205.7800     206.99  103372367.0   

               SPY_trade_count    SPY_vwap      TR  VIX_open  VIX_high  \
year week day                                                            
2015 49   3           367013.0  209.563055  2.7700     15.04     16.49   
          4           546768.0  206.878936  4.3989     15.87     19.35   
          5           556731.0  208.178631  4.3900     17.43     17.65   
     50   1           374705.0  208.276128  2.5295     15.65     17.18   
          2           387782.0  206.966276  2.5090     17.69     18.33   

               VIX_low  VIX_close  
year week day                      
2015 49   3      14.71      15.91  
          4      15.86      18.11  
          5      14.69      14.81  
     50   1      15.58      15.84  
          2      16.52      17.60

### Testing for get_array method

In [184]:
wk_grp_ind = week_group.index.to_list()
week_numbers = [tup[1] for tup in wk_grp_ind]
year_numbers = set([tup[0] for tup in wk_grp_ind])


In [185]:
year_numbers = list(year_numbers)

In [186]:
year_numbers = sorted(list(year_numbers))

In [187]:
df_x.head()

SPY_open  SPY_high   SPY_low  SPY_close   SPY_volume  \
year week day                                                         
2015 49   3      210.60  211.0000  208.2300     208.54  108069059.0   
          4      208.90  209.1500  204.7511     205.58  166224154.0   
          5      206.10  209.9700  205.9300     209.66  192878747.0   
     50   1      209.20  209.7295  207.2000     208.27  102027111.0   
          2      206.48  208.2890  205.7800     206.99  103372367.0   

               SPY_trade_count    SPY_vwap      TR  VIX_open  VIX_high  \
year week day                                                            
2015 49   3           367013.0  209.563055  2.7700     15.04     16.49   
          4           546768.0  206.878936  4.3989     15.87     19.35   
          5           556731.0  208.178631  4.3900     17.43     17.65   
     50   1           374705.0  208.276128  2.5295     15.65     17.18   
          2           387782.0  206.966276  2.5090     17.69     18.33   

               VIX_low  VIX_close  
year week day                      
2015 49   3      14.71      15.91  
          4      15.86      18.11  
          5      14.69      14.81  
     50   1      15.58      15.84  
          2      16.52      17.60

### get_array method

In [216]:
def get_array(df_input):
    idx = pd.IndexSlice
    df_input_ind = df_input.index.to_list()
    week_numbers = [tup[1] for tup in df_input_ind ]
    year_numbers = set([tup[0] for tup in df_input_ind ])
    year_numbers = sorted(list(year_numbers))
    
    week_data = []

    for yr_num in year_numbers:
        for wk_num in week_numbers: 
            try:
                week_data.append(df_input.loc[idx[yr_num, wk_num, :]].values)
            except:
                continue
            
    return week_data


In [300]:
def get_array_test(df_input):
    idx = pd.IndexSlice
    df_input_ind = df_input.index.to_list()
    week_numbers = [tup[1] for tup in df_input_ind ]
    year_numbers = set([tup[0] for tup in df_input_ind ])
    year_numbers = sorted(list(year_numbers))
    
    week_data = []

    for yr_num in year_numbers:
        for wk_num in week_numbers:
            
                try:
                    num_days = len(week_group.loc[idx[yr_num, wk_num]].index.to_list())
                    if num_days==5:
                        week_data.append(df_input.loc[idx[yr_num, wk_num, :]].values)
                except:
                    continue
            
    return week_data

### Check for Nans in Target df. Shift so that target matches features

In [295]:
df_y_shifted = df_y[df_y.weekly_range != 0].shift(-1)
df_y_shifted.isna().sum()

weekly_range    1
dtype: int64

In [ ]:
# display(df_x.head(20))
# display(df_y[df_y.weekly_range != 0].head(7).shift(-1))

In [297]:
x_data = get_array(df_x)
y_data = get_array(df_y_shifted)


In [319]:
x_data_test = get_array_test(df_x)
y_data_test = get_array_test(df_y_shifted)

In [ ]:
x_data_test

In [ ]:
x_data_test = np.array(x_data_test)
x_data_test

In [ ]:
week_group = df.groupby(by = [df.index.isocalendar().year, df.index.isocalendar().week,df.index.isocalendar().day])
weekly_ind = week_group.groups
# last_day_of_week = []
# for tup in weekly_ind.keys():
#     if tup[1] == 5:
#         last_day_of_week.append(weekly_ind[tup])

weekly_ind

In [197]:
def flatten(list_of_lists):
        return [item for sublist in list_of_lists for item in sublist]
last_day_of_week = flatten(last_day_of_week)

In [198]:
last_day_of_week = pd.Series(last_day_of_week)

In [ ]:
last_day_of_week

In [200]:
df['DayOfWeek'] = df.index.day_name()

In [201]:
df

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range,DayOfWeek
date,,,,,,,,,,,,,,
2015-12-02,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055,2.7700,15.04,16.49,14.71,15.91,0,Wednesday
2015-12-03,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936,4.3989,15.87,19.35,15.86,18.11,0,Thursday
2015-12-04,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631,4.3900,17.43,17.65,14.69,14.81,0,Friday
2015-12-07,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128,2.5295,15.65,17.18,15.58,15.84,0,Monday
2015-12-08,206.48,208.2890,205.7800,206.99,103372367,387782,206.966276,2.5090,17.69,18.33,16.52,17.60,0,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-23,446.90,448.4900,443.7100,443.80,77982935,627281,446.058683,5.8800,23.04,24.03,22.64,23.57,0,Wednesday
2022-03-24,445.82,450.5000,444.7600,450.48,61411569,526865,447.609014,6.7000,23.61,23.79,21.49,21.67,0,Thursday
2022-03-25,451.09,452.9800,448.4300,452.69,77199328,661033,451.047221,4.5500,21.87,22.86,20.80,20.81,0,Friday


In [202]:
# week_high = 0  
# week_low = 999999
# for index, row in df.iterrows():
#     if df.loc[index, 'DayOfWeek'] == 'Monday':
#         week_high = df.loc[index,'SPY_high']
#         week_low = df.loc[index,'SPY_low']
#     else: 
#         if df.loc[index,'SPY_high']>week_high:
#             week_high=df.loc[index,'SPY_high']
#         if df.loc[index,'SPY_low']<week_low:
#             week_low=df.loc[index,'SPY_low']
#         if df.loc[index,'DayOfWeek'] == 'Friday':
#             df.loc[index,'weekly_range'] = week_high - week_low
    

In [ ]:
def window_data(df, window, feature_col_1, feature_col_2, target_col):
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_col_1:feature_col_2]
        target = df.iloc[(i + window), target_col]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [323]:
df_x.reset_index()

,year,week,day,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close
0,2015,49,3,210.60,211.0000,208.2300,208.54,108069059.0,367013.0,209.563055,2.7700,15.04,16.49,14.71,15.91
1,2015,49,4,208.90,209.1500,204.7511,205.58,166224154.0,546768.0,206.878936,4.3989,15.87,19.35,15.86,18.11
2,2015,49,5,206.10,209.9700,205.9300,209.66,192878747.0,556731.0,208.178631,4.3900,17.43,17.65,14.69,14.81
3,2015,50,1,209.20,209.7295,207.2000,208.27,102027111.0,374705.0,208.276128,2.5295,15.65,17.18,15.58,15.84
4,2015,50,2,206.48,208.2890,205.7800,206.99,103372367.0,387782.0,206.966276,2.5090,17.69,18.33,16.52,17.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,2022,12,3,446.90,448.4900,443.7100,443.80,77982935.0,627281.0,446.058683,5.8800,23.04,24.03,22.64,23.57
1588,2022,12,4,445.82,450.5000,444.7600,450.48,61411569.0,526865.0,447.609014,6.7000,23.61,23.79,21.49,21.67
1589,2022,12,5,451.09,452.9800,448.4300,452.69,77199328.0,661033.0,451.047221,4.5500,21.87,22.86,20.80,20.81
1590,2022,13,1,452.04,455.9100,450.0600,455.91,68356914.0,620868.0,453.294974,5.8500,22.14,23.33,19.54,19.63


In [360]:
def drop_off_weeks(df_input):
    idx = pd.IndexSlice
    df_input_ind = df_input.index.to_list()
    week_numbers = [tup[1] for tup in df_input_ind ]
    year_numbers = set([tup[0] for tup in df_input_ind ])
    year_numbers = sorted(list(year_numbers))
    df_copy = df_input.copy()

    for yr_num in year_numbers:
        for wk_num in week_numbers:
            
                try:
                    num_days = len(week_group.loc[idx[yr_num, wk_num]].index.to_list())
                    if num_days < 5:
                       df_copy.drop(labels = wk_num, level=1, axis=0, inplace= True)
                except:
                    continue  
    return df_copy.reset_index()

In [361]:
test_df = drop_off_weeks(week_group)

In [363]:
test_df.drop(columns=['year', 'week', 'day'], inplace= True)


In [371]:
test_df['weekly_range_test'] = test_df['weekly_range'].shift(-5)
test_df.dropna(inplace=True)

In [401]:
len(test_df[test_df.columns.to_list()[:-2]].values)

1050

In [402]:
test_df

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range,weekly_range_test
0,209.20,209.7295,207.20,208.27,102027111.0,374705.0,208.276128,2.5295,15.65,17.18,15.58,15.84,0.0000,0.00
1,206.48,208.2890,205.78,206.99,103372367.0,387782.0,206.966276,2.5090,17.69,18.33,16.52,17.60,0.0000,0.00
2,206.20,208.6800,204.18,205.33,162401537.0,586210.0,206.034646,4.5000,18.05,20.13,15.72,19.61,0.0000,0.00
3,205.44,207.4300,205.14,205.86,116128858.0,404992.0,206.102975,2.2900,19.25,19.72,18.13,19.34,0.0000,0.00
4,203.38,204.1400,201.51,201.88,211173305.0,669924.0,203.150102,4.3500,21.36,25.27,20.88,24.39,8.2195,9.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,431.58,432.3018,419.36,419.43,131262026.0,1390460.0,424.067609,12.9418,35.88,36.55,32.59,36.45,0.0000,0.00
1046,419.87,427.2100,415.12,416.25,158890009.0,1864071.0,419.755053,12.0900,36.19,37.52,32.78,35.13,0.0000,0.00
1047,425.16,429.5100,422.83,427.33,110176608.0,1079760.0,426.138376,13.2600,33.74,34.12,31.39,32.45,0.0000,0.00
1048,422.42,426.4300,420.44,425.48,91933914.0,891241.0,423.871044,6.8900,33.03,34.03,30.23,30.23,0.0000,0.00


In [398]:
testing_shape = np.reshape(test_df[test_df.columns.to_list()[:-2]].values, (210, 5, 12))

In [399]:
testing_shape

array([[[209.2   , 209.7295, 207.2   , ...,  17.18  ,  15.58  ,
          15.84  ],
        [206.48  , 208.289 , 205.78  , ...,  18.33  ,  16.52  ,
          17.6   ],
        [206.2   , 208.68  , 204.18  , ...,  20.13  ,  15.72  ,
          19.61  ],
        [205.44  , 207.43  , 205.14  , ...,  19.72  ,  18.13  ,
          19.34  ],
        [203.38  , 204.14  , 201.51  , ...,  25.27  ,  20.88  ,
          24.39  ]],

       [[193.05  , 193.41  , 189.82  , ...,  27.39  ,  23.83  ,
          24.3   ],
        [193.85  , 194.55  , 191.14  , ...,  23.93  ,  21.91  ,
          22.47  ],
        [194.53  , 194.86  , 188.38  , ...,  26.11  ,  21.44  ,
          25.22  ],
        [189.55  , 193.26  , 187.66  , ...,  26.28  ,  23.07  ,
          23.95  ],
        [186.79  , 188.76  , 185.52  , ...,  30.95  ,  26.67  ,
          27.02  ]],

       [[192.5   , 194.58  , 191.84  , ...,  23.66  ,  19.61  ,
          19.98  ],
        [191.9   , 191.97  , 189.54  , ...,  22.42  ,  21.06  ,
        

In [403]:
test_df[test_df['weekly_range_test'] != 0 ]['weekly_range_test']

4        9.3400
9        7.4800
14       7.2500
19       8.0200
24       5.4300
         ...   
1029    18.3100
1034    18.9400
1039    13.9000
1044    17.1818
1049    29.0700
Name: weekly_range_test, Length: 210, dtype: float64

In [350]:
def window_data(df, window, feature_col_1, feature_col_2, target_col):
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_col_1:feature_col_2]
        target = df.iloc[(i + window), target_col]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [374]:
X , y = window_data(test_df, 5, 0, 11, 12)

In [382]:
len(X)/5

209.0

In [386]:
X

array([[[209.2   , 209.7295, 207.2   , ...,  15.65  ,  17.18  ,
          15.58  ],
        [206.48  , 208.289 , 205.78  , ...,  17.69  ,  18.33  ,
          16.52  ],
        [206.2   , 208.68  , 204.18  , ...,  18.05  ,  20.13  ,
          15.72  ],
        [205.44  , 207.43  , 205.14  , ...,  19.25  ,  19.72  ,
          18.13  ],
        [203.38  , 204.14  , 201.51  , ...,  21.36  ,  25.27  ,
          20.88  ]],

       [[206.48  , 208.289 , 205.78  , ...,  17.69  ,  18.33  ,
          16.52  ],
        [206.2   , 208.68  , 204.18  , ...,  18.05  ,  20.13  ,
          15.72  ],
        [205.44  , 207.43  , 205.14  , ...,  19.25  ,  19.72  ,
          18.13  ],
        [203.38  , 204.14  , 201.51  , ...,  21.36  ,  25.27  ,
          20.88  ],
        [193.05  , 193.41  , 189.82  , ...,  25.58  ,  27.39  ,
          23.83  ]],

       [[206.2   , 208.68  , 204.18  , ...,  18.05  ,  20.13  ,
          15.72  ],
        [205.44  , 207.43  , 205.14  , ...,  19.25  ,  19.72  ,
        